In [1]:
# --- Kaggle cell 1 ----------------------------------------------------------
# Kaggle gives you 30 GB of disk, but only 9 GB of RAM for pip installs.
# Install the minimal set in one shot so Kaggle does not kill the kernel.
!pip install -q --disable-pip-version-check \
    torch transformers datasets accelerate peft bitsandbytes streamlit

# Optional: if you want to push the trained adapter to the HF Hub
# from huggingface_hub import notebook_login
# notebook_login()   # paste token when prompted

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [2]:
# --- Kaggle cell 2 ----------------------------------------------------------
import os, torch, warnings, gc
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, get_peft_model, PeftModel

warnings.filterwarnings("ignore")
torch.cuda.empty_cache()

MODEL_NAME   = "distilgpt2"
MAX_LEN      = 256
BATCH        = 8
EPOCHS       = 2          # 2 epochs finish in ~8 min on T4×1
LR           = 3e-4
OUTPUT_DIR   = "/kaggle/working/empathetic_bot"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# 1. Load & tokenize EmpatheticDialogues
raw_ds = load_dataset("empathetic_dialogues")

def format_example(ex):
    text = f"<|prompt|>{ex['prompt']}<|endoftext|><|response|>{ex['utterance']}"
    return tokenizer(text, truncation=True, max_length=MAX_LEN)

tokenized = raw_ds.map(format_example, remove_columns=raw_ds["train"].column_names)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# 2. Model + LoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
lora_config = LoraConfig(
    r=8, lora_alpha=32, target_modules=["c_attn"], lora_dropout=0.05, bias="none"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 3. Train
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,      # skip eval to save time
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=True,
    logging_steps=50,
    save_total_limit=1,
    report_to="none",
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# 4. Quick sanity chat test
pipe = pipeline(
    "text-generation",
    model=PeftModel.from_pretrained(
        AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16),
        OUTPUT_DIR
    ),
    tokenizer=tokenizer,
    device_map="auto"
)

def chat(user_text):
    prompt = f"<|prompt|>{user_text}<|endoftext|><|response|>"
    out = pipe(
        prompt,
        max_new_tokens=64,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]
    return out.split("<|response|>")[-1].strip()

print("💙 EmpathyBot ready! Try:")
for q in ["I feel overwhelmed with work.", "My cat is sick and I’m scared."]:
    print("You:", q)
    print("Bot:", chat(q), "\n")

2025-08-13 12:58:13.228739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755089893.423078      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755089893.476916      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.798300
100,2.909800
150,2.719200
200,2.625900
250,2.556400
300,2.596600
350,2.572700
400,2.537800
450,2.510300
500,2.501300


Device set to use cuda:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 

💙 EmpathyBot ready! Try:
You: I feel overwhelmed with work.
Bot: I feel overwhelmed with work.  I feel overwhelmed with work.  It's not like I'm a scientist.  That's not fun.  I'm just waiting for the results.  So I don't want to take the time off.  I'm just waiting for results.  So I don't want 

You: My cat is sick and I’m scared.
Bot: I am hoping it doesn't get too bad. I can't wait for it to get worse.   I am so scared.  I can't wait for it to get worse.    I am so scared.       I can't wait for it to get worse. 



In [3]:
# -----------------------------------------------------------
# Cell: CLI chat inside the same Kaggle notebook
# -----------------------------------------------------------
import sys, select, os, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

MODEL_ID   = "distilgpt2"
ADAPTER    = "/kaggle/working/empathetic_bot"
MAX_TOKENS = 64
TEMP       = 0.7

print("🔧 Loading tokenizer …")
tokenizer = AutoTokenizer.from_pretrained(ADAPTER)
tokenizer.pad_token = tokenizer.eos_token

print("🔧 Loading base model …")
base = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("🔧 Merging LoRA adapter …")
model = PeftModel.from_pretrained(base, ADAPTER)
model.eval()

def generate(prompt_text: str) -> str:
    prompt = f"<|prompt|>{prompt_text}<|endoftext|><|response|>"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(
            **inputs,
            max_new_tokens=MAX_TOKENS,
            temperature=TEMP,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    return text.split("<|response|>")[-1].strip()

print("\n💙 EmpathyBot CLI ready!  Type /quit to exit.\n")
try:
    while True:
        user = input("> ").strip()
        if not user or user == "/quit":
            print("👋 Good-bye!")
            break
        print("Bot:", generate(user), "\n")
except KeyboardInterrupt:
    print("\n👋 Good-bye!")

🔧 Loading tokenizer …
🔧 Loading base model …
🔧 Merging LoRA adapter …

💙 EmpathyBot CLI ready!  Type /quit to exit.



>  My cat is sick and I’m scared.


Bot: I hope you get help.  I know you will have a good time.   I hope you can come back at home with a little help. 



>  I feel overwhelmed with work


Bot: I feel overwhelmed with work  I feel overwhelmed with work  i think  and i feel good about it 


👋 Good-bye!
